In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-129779
Azure region: southcentralus
Subscription id: b4a122b5-b4d5-40e7-9878-57b87adf4a8b
Resource group: aml-quickstarts-129779


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

#name for cpu cluster
amlcompute_cluster_name = "cpu-cluster"

#check for existance of cluster
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, using it ...')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes = 4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
aml_compute.wait_for_completion(show_output=True)


Found existing cluster, using it ...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0,100),
        "--max_iter": choice([50, 100, 150, 200])
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=2)
import shutil
if "training" not in os.listdir():
    os.mkdir("./training")
shutil.copy('./train.py','./training')

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './training',
              compute_target=aml_compute,
              entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     max_total_runs=4,
                                     max_concurrent_runs=4,
                                     )

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperopt_run = exp.submit(hyperdrive_config, tag={'type':'hyperdrive_opt'})
RunDetails(hyperopt_run).show()
### YOUR CODE HERE ###

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [13]:
import joblib
# Get your best run and save the model from that run.
hyperopt_run.wait_for_completion(show_output=True)
assert (hyperopt_run.get_status() == "Completed")
best_run = hyperopt_run.get_best_run_by_primary_metric()

best_run.register_model(model_name="bankmarketing_hyperdrive", model_path="outputs/bankmarketing_hyperdrive.joblib")

RunId: HD_c61b4511-7b85-426d-b72b-865bf8897571
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_c61b4511-7b85-426d-b72b-865bf8897571?wsid=/subscriptions/b4a122b5-b4d5-40e7-9878-57b87adf4a8b/resourcegroups/aml-quickstarts-129779/workspaces/quick-starts-ws-129779

Execution Summary
RunId: HD_c61b4511-7b85-426d-b72b-865bf8897571
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_c61b4511-7b85-426d-b72b-865bf8897571?wsid=/subscriptions/b4a122b5-b4d5-40e7-9878-57b87adf4a8b/resourcegroups/aml-quickstarts-129779/workspaces/quick-starts-ws-129779



ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/bankmarketing_hyperdrive.joblib in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_f011bb4042cd2ea88eb225d88e33ada8823b5b35155e4529413cbb1e796576c8_d.txt', 'azureml-logs/65_job_prep-tvmps_f011bb4042cd2ea88eb225d88e33ada8823b5b35155e4529413cbb1e796576c8_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_f011bb4042cd2ea88eb225d88e33ada8823b5b35155e4529413cbb1e796576c8_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/102_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path outputs/bankmarketing_hyperdrive.joblib in the set of files uploaded to the run: ['azureml-log

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(url)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
x["y"] = y # add target to training data
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=x,
    label_column_name="y",
    n_cross_validations=5)

In [2]:
# Submit your automl run
automl_run = exp.submit(automl_config, tag={'type':'automl'})
### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.
automl_run.wait_for_completion()
assert (automl_run.get_status() == "Completed")
best_run, best_model = automl_run.get_output()
joblib.dump(best_model, filename="automl_model.joblib")
### YOUR CODE HERE ###